In [3]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.9.tar.gz (384 kB)
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp39-cp39-macosx_10_9_universal2.whl size=244930 sha256=745dc8f7749ec81b1071c0987df1e2dd4b86bba849621799ea1c92e7654a90af
  Stored in directory: /Users/roger/Library/Caches/pip/wheels/3a/06/25/adb124afd8c8346e45c455f6586f7289cde2b4e339dfbcd9e9
Successfully built psycopg2
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sodapy import Socrata
import os
import numpy as np
import sqlalchemy as sa
from sqlalchemy import create_engine, text

pd.set_option('display.max_columns', None)

In [2]:
data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='erm2-nwe9'    # The data set at the API endpoint (311 data in this case)
app_token='kLUfNhl5SOgSzVFOKRG7nWfxZ'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds    
client.timeout = 6000

In [3]:
NYC_DATA_311 = "erm2-nwe9"
NYC_DATA_TREES = "5rq2-4hqu"

## Part 1: Data Preprocessing

### Download Dataset

In [4]:
directory='data'
# check is exist
if not os.path.exists(directory):
    # no exist then mkdir
    os.makedirs(directory)

In [5]:
# # # ### 311 Data  79.1w
results1 = client.get(NYC_DATA_311,limit=1500000)
df1 = pd.DataFrame.from_records(results1)
file1_name='my_311_data.csv'
df1.to_csv(f'{directory}/{file1_name}', index=False)

In [6]:
# # ### Tree Data 
results2 = client.get(NYC_DATA_TREES,limit=1500000)
df2 = pd.DataFrame.from_records(results2)
file2_name='my_trees_data.csv'
df2.to_csv(f'{directory}/{file2_name}', index=False)

### 311 complaints

In [7]:
#load complaints data 
data1=pd.read_csv('./data/my_311_data.csv',low_memory=False)

In [8]:
data1.head()

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,address_type,city,status,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_7mpf_4k6g,facility_type,due_date,intersection_street_1,intersection_street_2,landmark,vehicle_type,taxi_company_borough,taxi_pick_up_location,closed_date,bridge_highway_name,bridge_highway_direction,bridge_highway_segment,road_ramp
0,59681385,2023-12-09T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11222.0,452 MEEKER AVENUE,MEEKER AVENUE,RICHARDSON STREET,GRAHAM AVENUE,ADDRESS,BROOKLYN,Open,If the abandoned vehicle meets the criteria to...,2023-12-09T12:00:00.000,01 BROOKLYN,3.027250e+09,BROOKLYN,999359.0,201282.0,PHONE,Unspecified,BROOKLYN,40.719136,-73.945494,"{'latitude': '40.71913641790782', 'longitude':...",18182.0,36.0,2.0,30.0,57.0,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59682706,2023-12-09T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11412.0,114-03 197 STREET,197 STREET,MURDOCK AVENUE,115 AVENUE,ADDRESS,SAINT ALBANS,Open,If the abandoned vehicle meets the criteria to...,2023-12-09T12:00:00.000,12 QUEENS,4.110140e+09,QUEENS,1051577.0,194027.0,PHONE,Unspecified,QUEENS,40.698980,-73.757192,"{'latitude': '40.69897994741009', 'longitude':...",15312.0,41.0,3.0,6.0,71.0,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59683999,2023-12-09T12:00:00.000,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,11357.0,143-38 22 AVENUE,22 AVENUE,WHITESTONE EXPRESSWAY,144 STREET,ADDRESS,WHITESTONE,Open,If the abandoned vehicle meets the criteria to...,2023-12-09T12:00:00.000,07 QUEENS,4.042200e+09,QUEENS,1032666.0,223341.0,PHONE,Unspecified,QUEENS,40.779563,-73.825181,"{'latitude': '40.779563107109084', 'longitude'...",14192.0,22.0,3.0,20.0,67.0,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59681790,2023-12-09T02:41:46.000,DSNY,Department of Sanitation,Graffiti,Graffiti,Comercial,10032.0,603 WEST 161 STREET,WEST 161 STREET,NaN,NaN,ADDRESS,NEW YORK,Open,The graffiti on this property has been schedul...,2023-12-09T02:41:46.000,12 MANHATTAN,1.021370e+09,MANHATTAN,999919.0,244116.0,UNKNOWN,Unspecified,MANHATTAN,40.836703,-73.943374,"{'latitude': '40.83670343588696', 'longitude':...",13090.0,47.0,4.0,23.0,21.0,21.0,NaN,2024-01-08T02:41:46.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59684401,2023-12-09T02:06:35.000,DSNY,Department of Sanitation,Graffiti,Graffiti,Residential,11211.0,436 UNION AVENUE,UNION AVENUE,NaN,NaN,ADDRESS,BROOKLYN,Open,The graffiti has been reported. The property o...,2023-12-09T02:06:35.000,01 BROOKLYN,3.027610e+09,BROOKLYN,997692.0,199196.0,UNKNOWN,Unspecified,BROOKLYN,40.713414,-73.951512,"{'latitude': '40.71341353837136', 'longitude':...",17613.0,36.0,2.0,30.0,56.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# data1.isnull().sum()

In [10]:
print(data1.columns.tolist())

['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type', 'descriptor', 'location_type', 'incident_zip', 'incident_address', 'street_name', 'cross_street_1', 'cross_street_2', 'address_type', 'city', 'status', 'resolution_description', 'resolution_action_updated_date', 'community_board', 'bbl', 'borough', 'x_coordinate_state_plane', 'y_coordinate_state_plane', 'open_data_channel_type', 'park_facility_name', 'park_borough', 'latitude', 'longitude', 'location', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih', ':@computed_region_7mpf_4k6g', 'facility_type', 'due_date', 'intersection_street_1', 'intersection_street_2', 'landmark', 'vehicle_type', 'taxi_company_borough', 'taxi_pick_up_location', 'closed_date', 'bridge_highway_name', 'bridge_highway_direction', 'bridge_highway_segment', 'road_ramp']


In [11]:
data1=data1.drop(columns=[':@computed_region_f5dn_yrer',":@computed_region_yeji_bk3q",":@computed_region_92fq_4b7q",":@computed_region_sbqj_enih",":@computed_region_7mpf_4k6g","bridge_highway_name","bridge_highway_direction","bridge_highway_segment","vehicle_type","facility_type","taxi_pick_up_location","road_ramp","due_date",
                          ":@computed_region_efsh_h5xi",":@computed_region_efsh_h5xi",":@computed_region_f5dn_yrer",'landmark','closed_date','intersection_street_1','intersection_street_2','location','resolution_description','bbl','resolution_action_updated_date','borough','open_data_channel_type',
                          'park_facility_name','park_borough','agency','agency_name','descriptor','address_type','community_board','status','cross_street_1','cross_street_2','x_coordinate_state_plane','y_coordinate_state_plane','incident_address','street_name'
                         
                         ])

#### Column astype

In [12]:
##created_date
data1['created_date']=pd.to_datetime(data1['created_date'])

In [13]:
##incident_zip
data1['incident_zip'] = data1['incident_zip'].replace([np.nan, np.inf, -np.inf], -1)
data1['incident_zip']=data1['incident_zip'].astype(int)

In [14]:
data1['geom'] = data1.apply(lambda row: f'POINT({row["latitude"]} {row["longitude"]})', axis=1)
data1.drop(columns=['latitude','longitude'],inplace=True)

In [15]:
data1.head()

,unique_key,created_date,complaint_type,location_type,incident_zip,city,taxi_company_borough,geom
0,59681385,2023-12-09 12:00:00,Derelict Vehicles,Street,11222,BROOKLYN,NaN,POINT(40.71913641790782 -73.94549446766062)
1,59682706,2023-12-09 12:00:00,Derelict Vehicles,Street,11412,SAINT ALBANS,NaN,POINT(40.69897994741009 -73.757191809592)
2,59683999,2023-12-09 12:00:00,Derelict Vehicles,Street,11357,WHITESTONE,NaN,POINT(40.77956310710909 -73.82518126884483)
3,59681790,2023-12-09 02:41:46,Graffiti,Comercial,10032,NEW YORK,NaN,POINT(40.83670343588696 -73.94337438646215)
4,59684401,2023-12-09 02:06:35,Graffiti,Residential,11211,BROOKLYN,NaN,POINT(40.71341353837136 -73.95151231386045)


### Tree data

In [16]:
data2=pd.read_csv('data/my_trees_data.csv')

In [17]:
data2.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trnk_wire,trnk_light,trnk_other,brnch_ligh,brnch_shoe,brnch_othe,address,zipcode,zip_city,cb_num,borocode,boroname,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.768749
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,None,None,Damage,TreesCount Staff,Stones,Yes,No,No,No,No,No,No,No,No,147-074 7 AVENUE,11357,Whitestone,407,4,Queens,19,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.837379
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,1or2,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,390 MORGAN AVENUE,11211,Brooklyn,301,3,Brooklyn,34,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.891267
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,1027 GRAND STREET,11211,Brooklyn,301,3,Brooklyn,34,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.253136
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,American linden,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,603 6 STREET,11215,Brooklyn,306,3,Brooklyn,39,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.425999


In [18]:
data2=data2.drop(columns=['block_id','the_geom','tree_dbh','stump_diam','curb_loc','spc_latin','spc_common','steward','guards',
                    'sidewalk','user_type','root_stone','root_grate','root_other','trnk_wire','trnk_light','brnch_shoe',
                    'brnch_othe','address','cb_num','borocode','boroname','cncldist','st_assem','st_senate','nta','nta_name',
                    'boro_ct','x_sp','y_sp','trnk_other','brnch_ligh'])

In [19]:
data2['geom'] = data2.apply(lambda row: f'POINT({row["latitude"]} {row["longitude"]})', axis=1)
data2.drop(columns=['latitude','longitude'],inplace=True)

In [20]:
data2.head()

,created_at,tree_id,status,health,problems,zipcode,zip_city,state,geom
0,08/27/2015,180683,Alive,Fair,None,11375,Forest Hills,New York,POINT(40.72309177 -73.84421522)
1,09/03/2015,200540,Alive,Fair,Stones,11357,Whitestone,New York,POINT(40.79411067 -73.81867946)
2,09/05/2015,204026,Alive,Good,None,11211,Brooklyn,New York,POINT(40.71758074 -73.9366077)
3,09/05/2015,204337,Alive,Good,Stones,11211,Brooklyn,New York,POINT(40.71353749 -73.93445616)
4,08/30/2015,189565,Alive,Good,Stones,11215,Brooklyn,New York,POINT(40.66677776 -73.97597938)


### zillow rent

In [21]:
data3=pd.read_csv('/Users/apple/2023 Fall 4501/Project/data/Zillow_rent_data.csv')

In [22]:
data3.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1606.206406,1612.779844,1622.201575,1630.392427,1632.411500,1636.206864,1644.894632,1643.390762,1636.971443,1620.756709,1613.330035,1595.875303,1587.956334,1571.722899,1580.226011,1577.360511,1575.019249,1570.437252,1560.570981,1559.473860,1550.038637,1532.579831,1510.242213,1486.489733,1507.802059,1533.972404,1560.775400,1560.921517,1569.898260,1578.146822,1578.913743,1567.921730,1577.355514,1590.264504,1606.499320,1604.421318,1611.899807,1618.293318,1621.761808,1617.780728,1614.801931,1615.532634,1611.448067,1601.793861,1590.638710,1590.488945,1594.393947,1598.275260,1604.520456,1610.434841,1621.770680,1625.351916,1626.773326,1625.936662,1624.890088,1633.134397,1625.910897,1629.721928,1626.974655,1636.612419,1640.770934,1647.007094,1651.109647,1657.091010,1649.138584,1645.635867,1637.868714,1649.266605,1662.133844,1665.130638,1674.053593,1679.239150,1688.863037,1690.829750,1692.870693,1713.964591,1744.244031,1792.325270,1827.718052,1856.436987,1872.532636,1884.990347,1910.473392,1917.549543,1930.611286,1926.224851,1936.273593,1950.552503,1975.971322,1999.610405,2023.300973,2024.379276,2018.999370,2006.725802,1990.684558,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1257.814660,1255.268025,1262.170452,1274.955754,1285.526052,1295.665673,1296.650395,1300.868081,1301.898486,1302.881427,1299.693120,1296.038652,1288.469114,1287.887883,1296.983247,1310.096909,1316.314107,1308.568281,1300.912474,1295.072347,1292.669100,1288.731946,1279.219942,1270.826144,1270.159851,1280.804521,1290.962297,1297.787179,1299.429207,1305.319483,1313.028808,1314.471319,1318.936586,1318.360343,1325.904220,1327.061006,1338.242460,1342.947774,1346.339152,1347.977879,1353.781015,1360.934194,1361.964965,1362.711392,1358.947439,1356.220524,1356.988632,1353.259235,1350.062265,1348.472443,1354.870417,1359.648555,1367.652345,1372.060549,1380.044029,1377.192680,1378.572673,1375.046953,1372.524410,1373.553574,1381.286565,1390.962381,1402.096689,1398.546295,1390.275169,1390.301160,1403.523253,1422.490142,1432.232355,1431.127611,1437.327132,1448.187834,1458.760896,1465.848888,1471.032905,1490.807492,1527.079852,1565.896231,1605.220723,1614.231294,1619.765081,1631.133342,1636.650862,1656.262010,1650.745900,1677.114958,1697.240420,1720.421368,1698.930423,1697.851631,1735.015389,1802.088342,1802.928430,1790.160022,1752.950850,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Ha

In [23]:
data3.shape

(6722, 114)

In [24]:
# row to column
data3=pd.melt(data3,id_vars=['RegionID','SizeRank','RegionName','RegionType','StateName','State','City','Metro','CountyName'],var_name='date',value_name='rent')

In [25]:
data3['CountyName']=data3['CountyName'].str.replace(' County','')
# data3=data3.rename(columns={'CountyName':'county'})
data3.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,date,rent
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend,2015-01-31,1606.206406
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris,2015-01-31,1257.814660
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris,2015-01-31,NaN
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso,2015-01-31,NaN
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens,2015-01-31,NaN


In [26]:
data3=data3.rename(columns={'CountyName':'county'})

In [27]:
data3.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,county,date,rent
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend,2015-01-31,1606.206406
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris,2015-01-31,1257.814660
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris,2015-01-31,NaN
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso,2015-01-31,NaN
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens,2015-01-31,NaN


In [28]:
## drop column
data3=data3.drop(columns=['SizeRank','RegionType','StateName','Metro','City'])

## NYC zipcode

In [29]:
import geopandas as gpd

In [30]:
gdf=gpd.read_file('/Users/apple/2023 Fall 4501/Project/data/nyc_zipcodes.shp')

In [31]:
gdf.columns

Index(['ZIPCODE', 'BLDGZIP', 'PO_NAME', 'POPULATION', 'AREA', 'STATE',
       'COUNTY', 'ST_FIPS', 'CTY_FIPS', 'URL', 'SHAPE_AREA', 'SHAPE_LEN',
       'geometry'],
      dtype='object')

In [32]:
# data4=pd.DataFrame(gdf)
data4=gdf
data4.head(4)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."


In [33]:
data4.drop(columns=['URL','ST_FIPS','SHAPE_AREA','SHAPE_LEN','BLDGZIP','AREA','CTY_FIPS'],inplace=True)
data4=data4.rename(columns={'geometry':'geom'})
data4['geom']=data4['geom'].apply(lambda x:x.wkt)

In [34]:
# data4.drop(columns=['geom'],inplace=True)

In [35]:
data4.head()

,ZIPCODE,PO_NAME,POPULATION,STATE,COUNTY,geom
0,11436,Jamaica,18681.0,NY,Queens,POLYGON ((1038098.2518714815 188138.3800067156...
1,11213,Brooklyn,62426.0,NY,Kings,POLYGON ((1001613.7129640579 186926.4395172149...
2,11212,Brooklyn,83866.0,NY,Kings,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,Brooklyn,56527.0,NY,Kings,POLYGON ((995908.3654508889 183617.61280155182...
4,11218,Brooklyn,72280.0,NY,Kings,"POLYGON ((991997.1134308875 176307.4958601296,..."


## Part 2: Storing Data

In [4]:
import psycopg2
from psycopg2 import OperationalError

def create_connection(db_password, db_name='postgres', db_user='postgres', db_host='localhost', db_port=5432):
    conn = None
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return conn

In [ ]:
conn=create_connection(db_password='rog')
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cur=conn.cursor()

### Create Database

In [ ]:
cur.execute('''
create database gis
''')
cur.close()
conn.close()

In [ ]:
conn=create_connection(db_password='rog',db_name='gis')
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cur=conn.cursor()
res=cur.execute('''
CREATE EXTENSION IF NOT EXISTS postgis;
''')
print(res)

In [ ]:
cur.close()
conn.close()

### Create Table

In [ ]:
conn=create_connection(db_name='gis',db_password='czw')
cur=conn.cursor()

In [ ]:
cur.execute('''

 CREATE TABLE complaints (
     unique_key SERIAL PRIMARY KEY,
     created_date TIMESTAMP WITHOUT TIME ZONE,
     location_type TEXT,
     incident_zip INTEGER,
     geom GEOMETRY(Point, 4326)
 );
 create table tree(
 created_date TIMESTAMP WITHOUT TIME ZONE,
     tree_id integer,
     status varchar(50),
     health varchar(50),
     problems text,
     zipcode varchar(10),
     state varchar(10),
     geom GEOMETRY(Point, 4326)
 );

 create table rent
 (
 RegionID integer,
 RegionName	varchar(100),
 date TIMESTAMP WITHOUT TIME ZONE,
 rent float
 );
 create Table zip_code
 (
 ZIPCODE varchar(50),
 PO_NAME varchar(50),
 POPULATION varchar(50),
 STATE varchar(50),
 COUNTY  varchar(50),
 geom GEOMETRY(Point, 4326)
 )
 ''')
cur.close()
conn.commit()
conn.close()

### DataFrame to Postgresql DB Table

In [ ]:
# db configure 
db_user = 'postgres'
db_password = 'czw'
db_host = 'localhost'
db_port = '5432' # 
db_name = 'gis'
# create SQLAlchemy 
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [ ]:
data1.head()

In [ ]:
data1.to_sql('complaints',engine,if_exists='replace',index=False)

In [ ]:
data2.to_sql('tree',engine,if_exists='replace',index=False)

In [ ]:
data3.to_sql('rent',engine,if_exists='replace',index=False)

In [ ]:
data4.to_sql('zip_code',engine,if_exists='replace',index=False)

## Part 3:

test=1